# 2 - Sudoku 

  O objetivo é preecncher uma grelha de *N²*$\times$*N²* com inteiros positivos no intervalo de 1 até *N²*, satisfazendo as seguintes regras:

  *   Cada inteiro no intervalo de 1 até *N²* ocorre só uma vez em cada coluna, linha e secção *N*$\times$*N*.
  *   No início do jogo uma fracção *0* $\leq \alpha <$ 1 das N⁴ casas da grelha são preenchidas de forma consistente com a regra anterior.

Pretende-se: 

  *   Construir um programa para inicializar a grelha a partir dos parâmetros N e $\alpha$.
  *   Construir soluções do problema paras as combinações de parâmeteos *N* $\in$ {3,4,5,6} e $\alpha \in$ {0.0, 0.2, 0.4, 0.6} e tirar conclusões da complexidade computacional destas soluções.

**Análise do Problema** 

  É esperado receber um *N* e um $\alpha$, por parte do utilizador, onde *N* corresponde ao tamanho de cada secção do Sudoku que será representado e $\alpha$ a fracção de casas já preenchidas pela máquina. 

  Deste modo, o método utilizado para a construção do Sudoku foi a de um grafo. A estrutura do grafo tem a seguinte forma, *(n, A)*, onde *n* corresponde ao nodo e *A* corresponde há lista de nodos que têm uma relação com ele, ou seja, 
pertemcem à mesma secção, linha ou coluna. 

Seguidamente, a função **ip_color2** que verifica todas as relações entre os nodos e atribui uma cor diferente para nodos adjacentes. Deste modo, não é possivel existir a mesma cor numa mesma secção, linha ou coluna. 

Finalmente, como cada cor é lhe atribuído um número $\varkappa \in$*N²*. O grafo é representado em forma de matriz com cada cor do nodo a representado como um número.    










In [1]:
!pip install ortools 

In [2]:
import numpy as np
import math 
import networkx as nx
import time
from random import randint
import random 
from ortools.linear_solver import pywraplp

In [3]:
def checkSquare(x1,y1,x2,y2,size):
        if x1//size == x2//size and y1//size == y2//size:
            return True
        return False

def createEdges(mat):
    adj=[]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for outros in range(x,len(mat[y])):
                adj.append((mat[y][x],mat[y][outros]))
    for y in range(len(mat)):
        for x in range(len(mat[x])):
            for outros in range(y,len(mat)):
                adj.append((mat[y][x],mat[outros][x]))
            
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for yy in range(len(mat)):
                for xx in range(len(mat[yy])):
                    if checkSquare(x,y,xx,yy,int(math.sqrt(len(mat)))) and (mat[y][x],mat[yy][xx]) not in adj:
                        adj.append((mat[y][x],mat[yy][xx]))
    '''
    for (x,y) in adj:
        if (y,x) is adj:
            adj.remove((y,x))
    adj = list(set(adj))
    for (x,y) in adj:
        if x==y :
            adj.remove((x,y))'''
    adjq = [(x,y) for (x,y) in adj if x!=y and (y,x)]
    return adjq

In [19]:
def ip_color2(graph,k):
    

    # criar solver
    solver = pywraplp.Solver('BOP', pywraplp.Solver.BOP_INTEGER_PROGRAMMING)
    #criar dicionario de variaveis x{i,j} 
    x = {}
    for i in graph:
        x[i] = {}
        for j in range(k):
            x[i][j] = solver.IntVar(0,1,'x[%i][%i]' % (i,j))

    # vertices pre-preenchidos 
    for o in graph:
        if graph.nodes[o]['color'] != 0: 
              c = graph.nodes[o]['color']
              solver.Add(x[o][c-1] == 1)

      
           
    my_list = []
      
    # vertices adjacentes tem cores diferentes
    for o in graph:
      for d in graph[o]:
            for j in range(k):
                  if my_list.count(d) < 1:
                      solver.Add(x[o][j] + x[d][j] <= 1)
    
      my_list.append(o)         
                  
            
    
    # cada vertice adjacente tem cores diferentes

    for i in graph:
        solver.Add(sum([x[i][j] for j in range(k)]) == 1)  # ou solver.Add(sum(list(x[i].values())) == 1) 

    # invocar solver e colorir o grafo

    stat = solver.Solve()
    if stat == pywraplp.Solver.OPTIMAL:

      # colorir

        for i in graph:
            for j in range(k):
                if round(x[i][j].solution_value()) == 1:
                    graph.nodes[i]['color'] = j+1
        return True
    else:
        return False

In [31]:
def print_2_graph(mat, graph): 
    for row in range( len(mat) ):
      for col in range( len(mat[row]) ):
          id_node = row*len(mat) + col + 1
          graph.nodes[id_node]['color'] = mat[row][col]

In [38]:
def converte(grafo, size):
    cores=[]
    #value = randint(0, size)
    for i in grafo:
        cores.append(grafo.nodes[i]['color'])

    #for i in range(len(cores)):
        #cores[i] = ((cores[i] + value) % size) + 1
    solvedSudoku=np.reshape(cores,(int(math.sqrt(len(cores))),int(math.sqrt(len(cores)))))

    #solvedSudoku = apagaMat(solvedSudoku, prob)
    return solvedSudoku

In [6]:
def probCheck(prob):
    n = random.random()
    if(n <= prob):
        return True 
    return False 

def apagaMat(mat, prob): 
    prob = 1 - prob
    for y in range(len(mat)): 
        for x in range(len(mat[y])): 
            if probCheck(prob): 
                mat[y][x] = 0
    return mat 

In [36]:
def validaPos(mat,x,y,n):
    tam = len(mat)
    for xver in range(tam):
        if mat[y][xver] == n:
            return False
    for yver in range(tam):
        if mat[yver][x] == n:
            return False
    sqSize = int(math.sqrt(tam))
    # Check Square
    inicioLin = y - y % sqSize
    inicioCol = x - x % sqSize
    for i in range(sqSize):
        for j in range(sqSize):
            if mat[i + inicioLin][j + inicioCol] == n:
                return False
    return True

def geraMat(tam,prob):
    size = pow(tam,2)
    mat = [[0 for i in range(size)] for j in range(size)]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            if probCheck(prob):
                num = random.randint(1, size)
                if(validaPos(mat,x,y,num)):
                    mat[y][x] = num
    return mat

In [53]:
def checkZero(mat):
    for y in range( len(mat) ):
        for x in range( len(mat[y]) ):
            if mat[y][x]==0:
                return False
    return True

Execução do código para n = 3.


In [88]:
pontos = [x+1 for x in range(9**2)]
pontos = np.reshape(pontos,(9,9)) 
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
mat = geraMat(3, 0.6)

for x in mat: 
    print(x)
print('\n')

print_2_graph(mat, grafo5)
ti = time.perf_counter()
ip_color2(grafo5,9)
tf = time.perf_counter()
solved = converte(grafo5, 9)
if checkZero(solved):
    print(solved)

else: print("Não há solução.")


print(f"Time {tf - ti: 0.5f} seconds")


[0, 0, 0, 0, 0, 0, 8, 4, 0]
[0, 1, 4, 3, 0, 9, 2, 0, 0]
[0, 8, 0, 0, 0, 0, 6, 5, 0]
[6, 0, 2, 0, 0, 0, 1, 0, 9]
[3, 0, 0, 4, 0, 1, 0, 0, 0]
[8, 7, 0, 6, 0, 0, 0, 2, 0]
[0, 4, 0, 0, 8, 6, 5, 9, 0]
[0, 6, 3, 0, 0, 4, 0, 7, 1]
[0, 2, 0, 0, 0, 0, 0, 0, 0]


Não há solução.
Time  0.13042 seconds


Execução do código para n = 4.

In [92]:
pontos = [x+1 for x in range(16**2)]
pontos = np.reshape(pontos,(16,16))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
mat = geraMat(4, 0.4)

for x in mat: 
    print(x)
print('\n')

print_2_graph(mat, grafo5)
ti = time.perf_counter()
ip_color2(grafo5,16)
tf = time.perf_counter()
print(converte(grafo5, 16))
print(f"Time {tf - ti: 0.5f} seconds")

[0, 5, 0, 0, 0, 0, 16, 0, 0, 0, 14, 13, 8, 2, 4, 0]
[13, 2, 3, 0, 12, 0, 0, 0, 0, 0, 0, 1, 0, 14, 0, 5]
[8, 7, 0, 0, 0, 0, 0, 0, 0, 9, 5, 0, 0, 1, 0, 16]
[0, 11, 0, 0, 6, 0, 8, 0, 0, 0, 0, 10, 0, 0, 0, 0]
[0, 0, 4, 0, 0, 0, 6, 7, 0, 0, 0, 0, 14, 0, 0, 1]
[0, 0, 0, 12, 2, 0, 0, 0, 0, 1, 0, 14, 0, 0, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8, 0, 2, 11, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 13, 0, 0, 0, 0, 0, 7]
[5, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0]
[14, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 13, 0, 0, 0, 7, 0, 9, 6, 16, 0, 0, 0, 0, 10]
[9, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0]
[15, 0, 2, 0, 0, 0, 0, 16, 0, 0, 0, 0, 12, 0, 0, 0]
[0, 0, 0, 5, 0, 0, 2, 12, 0, 0, 0, 0, 6, 0, 0, 0]
[0, 0, 16, 7, 15, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 13, 0, 0]


[[ 6  5 15  9 10  3 16  1  7 11 14 13  8  2  4 12]
 [13  2  3 10 12  7  4 15  6 16  8  1 11 14  9  5]
 [ 8  7 12  4 14 11 13  2  3  9  5 15 10  1  6 16]
 [ 1 11 14 16  6  5  8  9 12  2  4

Execução do código para n = 5.

In [95]:
pontos = [x+1 for x in range(25**2)]
pontos = np.reshape(pontos,(25,25))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
mat = geraMat(5, 0.4)

for x in mat: 
    print(x)
print('\n')

print_2_graph(mat, grafo5)
ti = time.perf_counter()
ip_color2(grafo5,25)
tf = time.perf_counter()
print(converte(grafo5, 25))
print(f"Time {tf - ti: 0.5f} seconds")

[19, 0, 9, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 23, 0, 0, 0, 0, 0]
[0, 23, 0, 0, 3, 19, 0, 0, 0, 2, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 9, 15, 0, 16, 0]
[12, 14, 25, 20, 0, 0, 0, 8, 13, 17, 24, 0, 0, 15, 0, 18, 0, 22, 6, 10, 0, 0, 0, 0, 0]
[22, 0, 0, 0, 0, 0, 20, 0, 25, 0, 6, 0, 12, 23, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0]
[0, 0, 0, 0, 0, 21, 0, 4, 0, 0, 16, 0, 0, 0, 0, 5, 9, 0, 11, 0, 13, 18, 0, 0, 0]
[23, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 17, 19, 0, 0, 18, 0, 0, 0, 0, 0, 2, 0, 16]
[0, 25, 0, 0, 5, 0, 0, 23, 0, 0, 0, 0, 0, 0, 0, 17, 14, 0, 0, 0, 0, 8, 0, 0, 0]
[0, 0, 0, 6, 0, 0, 0, 13, 0, 5, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 14, 0]
[10, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 12, 0, 0, 0, 11, 0, 0, 0, 0, 25, 0, 0, 0, 0]
[0, 15, 0, 0, 0, 0, 24, 20, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 18, 0, 1, 0, 0, 5, 0, 0, 8, 15, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 23]
[0, 0, 19, 12, 11, 0, 0, 0, 14, 24, 0, 0, 6, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 18, 0]
[0, 17, 15, 0, 2, 0, 0, 0, 0, 

Execução do código para n = 6.

In [ ]:
pontos = [x+1 for x in range(36**2)]
pontos = np.reshape(pontos,(36,36))
edges=createEdges(pontos)
grafo5=nx.Graph(edges)
mat = geraMat(5, 0.4)

for x in mat: 
    print(x)
print('\n')

print_2_graph(mat, grafo5)
ti = time.perf_counter()
ip_color2(grafo5,36)
tf = time.perf_counter()
print(converte(grafo5))
print(f"Time {tf - ti: 0.5f} seconds")